# HW1 - computer vision IDC 2020/2021

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve2d
from scipy import signal

# specify the way plots behave in jupyter notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
plt.rcParams['image.cmap'] = 'gray'


## Section A: Convolution

In [2]:
def convolutionMaskA(img):
    mask = np.ones((1, 9)) / 9
    resultConv = convolve2d(img, mask, mode='same')
    absResultConv = abs(resultConv)
    f, ((ax1, ax2)) = plt.subplots(1, 2, sharex='col', sharey='row')
    ax1.imshow(img, cmap='gray'), ax1.set_title('Original Image')
    ax2.imshow(absResultConv, cmap='gray'), ax2.set_title('Applying convolution mask')
    plt.show()


def convolutionMaskB(img):
    mask = np.ones((5, 5)) * 1  # intesify all the 255 pixels, and weakens the L shape to a sum of 1
    mask[1, 1] = -5
    mask[2, 1] = -5
    mask[3, 1] = -5
    mask[3, 2] = -5
    mask = np.flip(mask)
    resultConv = convolve2d(img, mask, mode='same')
    f, ((ax1, ax2)) = plt.subplots(1, 2, sharex='col', sharey='row')
    ax1.imshow(img, cmap='gray'), ax1.set_title('Original Image')
    ax2.imshow(resultConv, cmap='gray'), ax2.set_title('Applying convolution mask')
    plt.show()


def convolutionMaskC(img):
    mask = np.ones((5, 5)) * 1  # intesify all the 255 pixels, and weakens the L shape to a sum of 1
    mask[1, 1] = -5
    mask[2, 1] = -4
    mask[3, 1] = -5
    mask[3, 2] = -5
    mask[2, 2] = 0  # the 'i dont care' value get a weight of 0 meaning it can be any value - it wont matter because he wont be weighted
    mask = np.flip(mask)
    resultConv = convolve2d(img, mask, mode='same')
    f, ((ax1, ax2)) = plt.subplots(1, 2, sharex='col', sharey='row')
    ax1.imshow(img, cmap='gray'), ax1.set_title('Original Image')
    ax2.imshow(resultConv, cmap='gray'), ax2.set_title('Applying convolution mask')
    plt.show()
    

img1=cv2.imread('cameraman.jpg', cv2.IMREAD_GRAYSCALE)
convolutionMaskA(img1)
convolutionMaskB(img1)
convolutionMaskC(img1)
img1=cv2.imread('synthCheck.tif', cv2.IMREAD_GRAYSCALE)
convolutionMaskA(img1)
convolutionMaskB(img1)
convolutionMaskC(img1)

## Section B: Canny Edge Detector

In [3]:
def mat_base(sigma):
    mask_size = sigma * 4
    if type(sigma) != int:
        mask_size = int(np.ceil(mask_size))
    # in order to get 95% signal we have to use 2 standard deviations from each side of the gauss bell
    if (mask_size % 2) == 0:  # check if the size is even
        mask_size += 1  # if it does -> make odd. by +1 we maintain 95% of the signal
    ax = np.linspace(-mask_size // 2, mask_size // 2, mask_size)
    [xx, yy] = np.meshgrid(ax,
                           ax)  # Assign two matrices with the appropriate x and y values on which the gaussian function is computed
    return xx, yy


def Deriv_Gauss_x(sigma):
    [x, y] = mat_base(sigma)
    G_dx = (-x / (2 * np.pi * sigma ** 4)) * np.exp(-(np.square(x) + np.square(y)) / (2 * sigma ** 2))
    return G_dx / G_dx.sum()


def Deriv_Gauss_y(sigma):
    [x, y] = mat_base(sigma)
    G_dy = (-y / (2 * np.pi * sigma ** 4)) * np.exp(-(np.square(x) + np.square(y)) / (2 * sigma ** 2))
    return G_dy / G_dy.sum()


def Grad_x(img, G_dx):
    resultConv = convolve2d(img, G_dx, mode='same')
    return resultConv


def Grad_y(img, G_dy):
    resultConv = convolve2d(img, G_dy, mode='same')
    return resultConv


def Grad_o(Ix, Iy):
    G_orientation = np.arctan2(Iy, Ix) * 180 / np.pi
    G_orientation[G_orientation < 0] += 180
    return G_orientation


def Grad_m(Ix, Iy):
    grad = (Ix ** 2 + Iy ** 2) ** 0.5
    grad *= 255.0 / grad.max()
    return grad


def thinning(G_magnitute, G_orientation):
    # first we'll divide orientation to four groups :0,45,90,135
    nms = np.zeros(G_magnitute.shape)
    mask1 = G_orientation < 22.5
    mask2 = G_orientation >= 157.5
    G_orientation[mask1 | mask2] = 0
    mask3 = G_orientation >= 22.5
    mask4 = G_orientation < 67.5
    G_orientation[mask3 & mask4] = 45
    mask5 = G_orientation >= 67.5
    mask6 = G_orientation < 112.5
    G_orientation[mask5 & mask6] = 90
    mask7 = G_orientation >= 112.5
    mask8 = G_orientation < 157.5
    G_orientation[mask7 & mask8] = 135
    pre = 0
    post = 0
    for i in range(1, G_magnitute.shape[0] - 1):
        for j in range(1, G_magnitute.shape[1] - 1):
            if G_orientation[i][j] == 0:
                pre = G_magnitute[i][j - 1]
                post = G_magnitute[i][j + 1]
            elif G_orientation[i][j] == 45:
                pre = G_magnitute[i - 1][j - 1]
                post = G_magnitute[i + 1][j + 1]
            elif G_orientation[i][j] == 90:
                pre = G_magnitute[i - 1][j]
                post = G_magnitute[i + 1][j]
            else:  # G_orientation[i][j] is 135
                pre = G_magnitute[i + 1][j - 1]
                post = G_magnitute[i - 1][j + 1]
            if G_magnitute[i][j] >= pre and G_magnitute[i][j] >= post:
                nms[i][j] = G_magnitute[i][j]
    return nms


def canny(img, sigma, L_th, H_th):
    G_dx = Deriv_Gauss_x(sigma);
    G_dy = Deriv_Gauss_y(sigma)
    Ix = Grad_x(img, G_dx);
    Iy = Grad_y(img, G_dy)
    G_orientation = Grad_o(Ix, Iy);
    G_magnitude = Grad_m(Ix, Iy)
    nms = thinning(G_magnitude, G_orientation)
    if L_th < 1 and H_th <= 1:  # meaning we are receiving a ratio of threshold and not an explicit threshold
        L_th = np.min(nms) + (np.max(nms) - np.min(nms)) * L_th
        H_th = np.min(nms) + (np.max(nms) - np.min(nms)) * H_th
    i_h = nms.copy().astype(np.uint8);
    i_h[nms < H_th] = 0
    i_l = nms.copy().astype(np.uint8);
    i_l[(H_th < nms) | (nms < L_th)] = 0
    kernel = np.ones((2, 2), np.uint8)
    i_h_d = cv2.dilate(i_h, kernel)
    labels_l, connected_mat_l = cv2.connectedComponents(i_l, connectivity=8)
    keys = np.unique(connected_mat_l[np.where((connected_mat_l > 0) & (i_h_d > 0))])
    edges = np.zeros(i_h.shape).astype(np.uint8)
    for item in keys:
        i, j = np.where(connected_mat_l == item)
        edges[i, j] = item
    i_h += edges
    i_h[i_h > 0] = 1  # binary map
    return i_h


def fig_canny(img, resultCanny):
    f, ((ax1, ax2)) = plt.subplots(1, 2, sharex='col', sharey='row')
    ax1.imshow(img, cmap='gray'), ax1.set_title('Original Image')
    ax2.imshow(resultCanny, cmap='gray'), ax2.set_title('Applying Canny edge detector')
    plt.show()
    cv2.waitKey(0)


img = cv2.imread('Church.jpg', cv2.IMREAD_GRAYSCALE)
resultCanny = canny(img, 1.3, 0.1, 0.15)
fig_canny(img, resultCanny)

In [4]:
imageName = 'Church_GT.bmp'
imgGT = cv2.imread(imageName, cv2.IMREAD_GRAYSCALE)
def evaluate_edges(resultCanny, imgGT):
    e_gt = np.sum(resultCanny + imgGT == 2)
    e = np.sum(resultCanny)
    gt = np.sum(imgGT)
    if gt == 0 and e == 0: #when both are equal 0, in theory the answer should be a perfect match. but then again 0 in both means no predictions so no reason to analyze?
        p =0
        r = 0
        f = 0
        return p,r,f  # no edges were detected at all so no true positives values
    if e == 0: # the comparison to the ground truth doesnt make any sense since there wasn't any prediction
        p = 0
        r = 0
        f = 0
        return p, r, f
    if gt == 0:
        p = 0
        r = 1  # no edges in the ground truth at all so all the predictions are false positive
        f = 0
        return p, r, f     
    p = round(e_gt / e, 3)
    r = round(e_gt / gt, 3)
    f = round(2 * p * r / (p + r), 3)
    return p, r, f
p,r,f = evaluate_edges(resultCanny, imgGT)
print("precision: {0}\nrecall: {1}\nf_value : {2}\n".format(p,r,f))

In [5]:
def gridsearch_parmeters(img_names, gt, sig, H_th, L_th):
    comb_arr = np.array(np.meshgrid(sig, L_th, H_th)).T.reshape(-1, len(sig));
    for i in range(len(img_names)):
        img = cv2.imread(img_names[i], 0);
        imgGT = cv2.imread(gt[i], cv2.IMREAD_GRAYSCALE);
        for comb in comb_arr:
            resultCanny = canny(img, comb[0], comb[1], comb[2]);
            p, r, f = evaluate_edges(resultCanny, imgGT);
            print("{0},parmeters:{1}\nprecision: {2}   recall: {3}   f_value :{4}\n{5}".format(img_names[i], comb, p, r,
                                                                                               f, '-' * 50));
            
img_names=['Church.jpg','Nuns.jpg','Golf.jpg']; gt=['Church_GT.bmp','Nuns_GT.bmp','Golf_GT.bmp'];
sig=[1,2,3];H_th=[0.13,0.15,0.18];L_th=[0.05,0.07,0.1];
gridsearch_parmeters(img_names,gt,sig,H_th,L_th);

In [6]:
def evaluate_edges_shift_tolerant(resultCanny, imgGT):
    e_gt = np.sum(resultCanny + imgGT == 2)
    e = np.sum(resultCanny)
    gt = np.sum(imgGT)
    canny = resultCanny.copy()  # making a working copy of cannys result matrix
    gt_shift = imgGT.copy()  # making a working copy GT in order to shift and erase used values
    '''START of basic evaluation without shifting:'''
    if gt == 0 and e == 0: #when both are equal 0, in theory the answer should be a perfect match. but then again 0 in both means no predictions so no reason to analyze?
        p =0
        r = 0
        f = 0
        return p,r,f  # no edges were detected at all so no true positives values
    if e == 0: # the comparison to the ground truth doesnt make any sense since there wasn't any prediction
        p =0
        r = 0
        f = 0
        return p,r,f
    if gt == 0:
        p =0
        r = 1 # no edges in the ground truth at all so all the predictions are false positive
        f = 0
        return p, r, f
    p = round(e_gt / e, 3)
    r = round(e_gt / gt, 3)
    f = round(2 * p * r / (p + r), 3)
    '''END  of basic evaluation without shifting:'''
    pd = 0
    shift_krnl = np.zeros((3, 3), np.uint8)
    list_of_kernels = []
    for i in range(3):  # Assign 8 cases of matrix shifting by 1 step into a list of kernels
        for j in range(3):
            if i == 1 and j == 1:
                continue
            shift_krnl[i, j] = 1
            list_of_kernels.append(shift_krnl)
            shift_krnl = np.zeros((3, 3), np.uint8)
    # first iteration: count and erase all the unshifted true positives from copied matrices
    e_gt_d = np.sum(canny + gt_shift == 2)
    i, j = np.where(canny + gt_shift == 2)
    canny[i, j] = 0
    gt_shift[i, j] = 0
    # start convolving the imgGT in all 8th directions
    for i in range(8):
        gt_shift = convolve2d(gt_shift, list_of_kernels[i], mode='same')  # shift the matrix into a picked direction
        e_gt_d += np.sum(canny + gt_shift == 2)  # if after shifting the matrix, matches were found -> count them
        # now erase all counted instances
        row, col = np.where(canny + gt_shift == 2)
        canny[row, col] = 0
        gt_shift[row, col] = 0
        # now return imgGT to its true form by passing the same kernel flipped
        gt_shift = convolve2d(gt_shift, np.flip(list_of_kernels[i]), mode='same')
    pd = round(e_gt_d / e, 3)
    if pd > p:
        rd = round(e_gt_d / gt, 3)
        fd = round(2 * pd * rd / (pd + rd), 3)
        return pd, rd, fd
    return p, r, f
print("RESULTS BEFORE SHIFTING:\n")
p,r,f = evaluate_edges(resultCanny,imgGT)
print("precision: {0}\nrecall: {1}\nf_value : {2}\n".format(p,r,f))
pd,rd,fd = evaluate_edges_shift_tolerant(resultCanny,imgGT)
print("RESULTS AFTER APPLYING SHIFTING\n")
print("precision: {0}\nrecall: {1}\nf_value : {2}\n".format(pd,rd,fd))